In [2]:
from exchangelib import Credentials, Account, Configuration, DELEGATE, Message, Mailbox, FileAttachment, HTMLBody
import os
import matplotlib.dates as mdates

In [3]:
credentials = Credentials('nldc\\00162', 'Sanju@809249')
config = Configuration(server='mail.grid-india.in', credentials=credentials)
account = Account(primary_smtp_address='sanjaykumar@grid-india.in', config=config,
                    autodiscover=False, access_type=DELEGATE)

In [10]:
m = Message(
        account=account,
        subject='Test',
        body='',
        to_recipients=[
            Mailbox(email_address='sanjaykumar@grid-india.in'),
        ],)

In [11]:
html_str = ''
th_str = ''
td_str = ''
forecast_html = ''

rmse_table=''
rmse_str = ''
html_body = '<html><body>\
                <p style="width:100%; padding:20px; background-color:#6495ed; color:#fff ">Forecast comparasion results for {} : </p><br/>\
                  {}\
                  {}\
                  {}<br/>\
                  <p style="width:100%; padding:20px; background-color:#6495ed; color:#fff "">Forecast for {} </p><br/>\
                  {}\
                  </body></html>'

m.body = HTMLBody(html_body)
m.send()

True

In [12]:
try:
    os.chdir("E:/forecast/load_forecast_v2")
except FileNotFoundError:
    os.chdir("E:/Projects/load_forecast/load_forecast_v2")
    
def sendDailyForecastMail(email_to):
    for_date = date.today()  - timedelta(days=0)
    compare_date = for_date - timedelta(days=1)
    forecast_date = for_date + timedelta(days=1)
    ##################################################################################################

    credentials = Credentials('nldc\\erldcloadforecast', 'Password1')
    config = Configuration(server='mail.posoco.in', credentials=credentials)
    account = Account(primary_smtp_address='erldcloadforecast@posoco.in', config=config,
                      autodiscover=False, access_type=DELEGATE)
    regions_map = {'BH_DEMAND': 'BIHAR', 'JH_DEMAND': 'JHARKHAND', 'DVC_DEMAND': 'DVC', 'GR_DEMAND': 'GRIDCO',\
                   'WB_DEMAND': 'WEST BENGAL', 'SI_DEMAND': 'SIKKIM', 'ER_DEMAND': 'EASTERN REGION'}

    m = Message(
        account=account,
        subject='ER Forecast for {} and comparison for {}'.format(forecast_date.strftime('%d-%m-%Y'), compare_date.strftime('%d-%m-%Y')),
        body='',
        to_recipients=[
            Mailbox(email_address=email_to),
        ],)

    region_list = ["BIHAR", "JHARKHAND", "DVC", "ODISHA", "WEST BENGAL", "SIKKIM", "EASTERN REGION"]
    COMPARE_PATH = '//10.3.100.24/HTTP-Access/ERForecast/forecast/{}/{}/{}'.format(compare_date.year, compare_date.strftime("%b"), compare_date.strftime("%d%m%Y"))
    FORECAST_PATH = '//10.3.100.24/HTTP-Access/ERForecast/forecast/{}/{}/{}'.format(forecast_date.year, forecast_date.strftime("%b"), forecast_date.strftime("%d%m%Y"))
    compare_file = 'fc_{}_{}.jpg'
    forecast_file = 'f_{}_{}.jpg'
    html_str = ''
    th_str = ''
    td_str = ''
    forecast_html = ''
    for region in region_list:
        region_file_name = compare_file.format(compare_date.strftime("%d%m%Y"), region)
        region_file_path = '{}/{}'.format(COMPARE_PATH, region_file_name)
        region_file_name = region_file_name.replace(" ", "_")
        with open(region_file_path, 'rb') as f:
            region_file = FileAttachment(name=region_file_name, content=f.read(), is_inline=True, content_id=region_file_name)
            html_str = html_str + '<img src="cid:{}">'.format(region_file_name)
            th_str = th_str + ' <th>' + region + '</th>'
            td_str = td_str + ' <td>' + str(1) + '</td>'
            m.attach(region_file)

    for region in region_list:
        region_file_name = forecast_file.format(forecast_date.strftime("%d%m%Y"), region)
        region_file_path = '{}/{}'.format(FORECAST_PATH, region_file_name)
        region_file_name = region_file_name.replace(" ", "_")
        with open(region_file_path, 'rb') as f:
            region_file = FileAttachment(name=region_file_name, content=f.read(), is_inline=True, content_id=region_file_name)
            forecast_html = forecast_html + '<img src="cid:{}">'.format(region_file_name)
            m.attach(region_file)

#     rmse_file_name = 'forecast/rmse_percent_{}.png'.format(compare_date.strftime("%d%m%Y"))
#     with open(rmse_file_name, 'rb') as f:
#         rmse_file = FileAttachment(name=rmse_file_name, content=f.read(), is_inline=True, content_id=rmse_file_name)
#         rmse_str = '<img src="cid:{}">'.format(rmse_file_name)
#         m.attach(rmse_file)
        
    forecast_compare_file_name = 'forecast_compare_{}.xlsx'.format(compare_date.strftime("%d%m%Y"))
    forecast_compare_file_path = '{}/{}'.format(COMPARE_PATH, forecast_compare_file_name)
    with open(forecast_compare_file_path, 'rb') as f:
        compare_file = FileAttachment(name=forecast_compare_file_name, content=f.read(), is_inline=False, content_id=forecast_compare_file_name)
        compare_str = '<img src="cid:{}">'.format(forecast_compare_file_name)
        m.attach(compare_file)
        
    forecast_file_name = 'forecast_{}.xlsx'.format(forecast_date.strftime("%d%m%Y"))
    forecast_file_path = '{}/{}'.format(FORECAST_PATH, forecast_file_name)
    with open(forecast_file_path, 'rb') as f:
        f_file = FileAttachment(name=forecast_file_name, content=f.read(), is_inline=False, content_id=forecast_file_name)
        f_str = '<img src="cid:{}">'.format(forecast_file_name)
        m.attach(f_file)
        


    rmse_table=''
    rmse_str = ''
    html_body = '<html><body>\
                    <p style="width:100%; padding:20px; background-color:#6495ed; color:#fff ">Forecast comparasion results for {} : </p><br/>\
                      {}\
                      {}\
                      {}<br/>\
                      <p style="width:100%; padding:20px; background-color:#6495ed; color:#fff "">Forecast for {} </p><br/>\
                      {}\
                      </body></html>'.format(compare_date.strftime("%d-%m-%Y"), rmse_table, rmse_str, html_str, forecast_date.strftime("%d-%m-%Y"), forecast_html)

    m.body = HTMLBody(html_body)
    m.send()
    print(plt.rcParams['figure.figsize'])
    return 0

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:/Projects/load_forecast/load_forecast_v2'

In [13]:
a=[1,2,3,4,5,6]

In [21]:
a[:-1],a[-1:]

([1, 2, 3, 4, 5], [6])

In [1]:
x= '3. Sanjay Kumar (00162) 2. Sanjay Kumar (00162) 1. Raju Biswas (raju) :Information Technology'

In [5]:
x= "Soumya Kanti Das (00277)"

In [8]:
x.split("(")[0][:-1]

'Soumya Kanti Das'

In [9]:
x.split("(")[1][:-1]

'00277'